# ANSWER KEY: Debug Drill 03 - Wrong Metric

**Bug:** Colleague optimized for accuracy on an imbalanced dataset where only 8% of customers churn.

**Key Lesson:** Accuracy is misleading for imbalanced problems. A model predicting "no one churns" gets 92% accuracy while being useless.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/data/streamcart_customers.csv')

## The Bug (Colleague's Code)

In [ ]:
# ===== BUGGY CODE =====
features = ['tenure_months', 'logins_last_30d', 'orders_last_30d', 'support_tickets_last_30d']
X = df[features].fillna(0)
y = df['churn_30d']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Colleague says: 'My model has {accuracy:.1%} accuracy! Ship it!'")

## Why This Is Wrong

In [ ]:
# Check the class balance
churn_rate = y.mean()
print(f"Class balance:")
print(f"  Churners: {churn_rate:.1%}")
print(f"  Non-churners: {1-churn_rate:.1%}")

# What's the "naive" accuracy?
naive_accuracy = 1 - churn_rate
print(f"\nNaive baseline (predict 'no churn' for everyone): {naive_accuracy:.1%} accuracy")
print(f"Colleague's model: {accuracy:.1%} accuracy")
print(f"\nThe model barely beats predicting 'no one churns'!")

In [ ]:
# What's the model actually predicting?
print(f"\nModel predictions:")
print(f"  Predicted churn: {y_pred.sum()}")
print(f"  Predicted no churn: {len(y_pred) - y_pred.sum()}")
print(f"  Actual churners in test: {y_test.sum()}")

if y_pred.sum() < 10:
    print("\nThe model is barely predicting anyone will churn!")
    print("It's essentially predicting the majority class for everyone.")

## The Fix

In [ ]:
# ===== FIXED CODE =====
# Use appropriate metrics for imbalanced data

y_proba = model.predict_proba(X_test)[:, 1]

print("Correct evaluation for imbalanced data:")
print("="*50)

# AUC - measures ranking ability
auc = roc_auc_score(y_test, y_proba)
print(f"AUC: {auc:.3f}")
print("  (0.5 = random, 1.0 = perfect ranking)")

# Precision@K - what matters if we can only call K customers
k = 100  # Assume we can call 100 customers
top_k_indices = np.argsort(y_proba)[::-1][:k]
precision_at_k = y_test.iloc[top_k_indices].mean()
print(f"\nPrecision@{k}: {precision_at_k:.1%}")
print(f"  (Base rate: {churn_rate:.1%})")
print(f"  (Lift: {precision_at_k/churn_rate:.1f}x vs random)")

In [ ]:
# Compare to business baseline
print("\nBusiness comparison:")
print(f"Random selection of 100 customers: ~{int(100*churn_rate)} churners")
print(f"Model's top 100 customers: ~{int(100*precision_at_k)} churners")
print(f"\nThe model finds {precision_at_k/churn_rate:.1f}x more churners than random!")

In [ ]:
# Self-check
assert auc > 0.55, "AUC should be better than random"
assert precision_at_k > churn_rate, "Model should beat baseline"
print("\nPASS: Using correct metrics for imbalanced data!")

## Completed Postmortem

### What happened:
- Colleague celebrated high accuracy on a dataset where 92% of customers don't churn
- The model learned to predict "no churn" for almost everyone (safe but useless)

### Root cause:
- Accuracy rewards predicting the majority class
- With 8% churn rate, predicting "no one churns" gives 92% accuracy automatically

### How to prevent:
- For imbalanced problems, use AUC, Precision@K, or Recall
- Always compare to a naive baseline (majority class prediction)
- Ask: "What would random selection achieve?" and ensure the model beats it